# IIACT: Ensemble for HMS Brain Comp by MLiP team
This is combination and ensemble notebook for Kaggle's HMS brain comp. based on [Previous Notebook](https://www.kaggle.com/code/luepoe/iiact-ensamble-features-head-starter) and [Thisone](https://www.kaggle.com/code/anilyagiz/hms-multiple-model-ensemble-4-notebooks-19b844) 


- https://www.kaggle.com/code/cdeotte/efficientnetb0-starter-lb-0-43 (That is the our model and our code)
- https://www.kaggle.com/code/yunsuxiaozi/hms-baseline-resnet34d-512-512-inference-6-models
- https://www.kaggle.com/code/andreasbis/hms-inference-lb-0-41
- https://www.kaggle.com/code/nartaa/features-head-starter-lb-0-36

Extra that needs to be added to this so its more vertasille:
- [LB 0.46] DilatedInception WaveNet - Inference (https://www.kaggle.com/code/abaojiang/lb-0-46-dilatedinception-wavenet-inference/notebook?scriptVersionId=163448688)
- CatBoost Starter - [LB 0.60]: [Notebook][https://www.kaggle.com/code/cdeotte/catboost-starter-lb-0-60] and our trained dataset on this

**The Ensemble achieves LB 0.34** 

Features+Head Starter uses Chris Deotte's Kaggle dataset [here][1]. Also Uses Chris's EEG spectrograms [here][3] (modified version) 

This notebook is based on the work of JIYUANZHANG, found [here](https://www.kaggle.com/code/kitsuha/3-model-ensemble-lb-0-37)

# Intro and Config

In [1]:
%pip install d2l --no-index --find-links=file:///kaggle/input/d2l-package/d2l/
%pip install /kaggle/input/brain-solver/brain_solver-0.9.0-py3-none-any.whl

Looking in links: file:///kaggle/input/d2l-package/d2l/
Note: you may need to restart the kernel to use updated packages.
Processing /kaggle/input/brain-solver/brain_solver-0.9.0-py3-none-any.whl
brain-solver is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Note: you may need to restart the kernel to use updated packages.


# Config Class Summary

The `Config` class manages configurations for a brain activity classification project. It includes:

- **Data and Model Paths**: Centralizes paths for data (e.g., EEG, spectrograms) and model checkpoints.
- **Training Parameters**: Configures training details like epochs, batch size, and learning rate.
- **Feature Flags**: Toggles for using wavelets, spectrograms, and reading options.

Designed for easy adjustments to facilitate model development and experimentation.

In [2]:
from brain_solver import Config

# full_path = "/home/osloup/NoodleNappers/brain/data/" # Luppo
# full_path = "C:/Users/tygof/Documents/Semester 8/MLiP/NoodleNappers/brain/data/" # Tygo
# full_path = "C:/Users/dahbl/Documents/TrueDocs/Uni/Year 4/Semester 2/Machine Learning in Practice/brain/brain/data/" # Dick
# config = Config(full_path,  full_path + "out/", USE_EEG_SPECTROGRAMS=True, USE_KAGGLE_SPECTROGRAMS=True, should_read_brain_spectograms=False, should_read_eeg_spectrogram_files=False, USE_PRETRAINED_MODEL=True, FINE_TUNE=True)

# Kaggle Pull
full_path = "/kaggle/input/"
config = Config(
    full_path,
    "/kaggle/working/",
    USE_EEG_SPECTROGRAMS=False,
    VER=5,
    USE_KAGGLE_SPECTROGRAMS=True,
    should_read_brain_spectograms=False,
    should_read_eeg_spectrogram_files=False,
    USE_PRETRAINED_MODEL=False,
)

import sys

sys.path.append(full_path + "kaggle-kl-div")
from kaggle_kl_div import score

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
import os
import pytorch_lightning as pl

# Create Output folder if does not exist
if not os.path.exists(config.output_path):
    os.makedirs(config.output_path)

# Initialize random environment
pl.seed_everything(config.seed, workers=True)

2024

# Model 1 - Starter/Ours

In [4]:
import os, sys
import gc
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from brain_solver import (
    Helpers as hp,
    Trainer as tr,
    BrainModel as br,
    EEGDataset,
    Network,
)
from brain_solver import Wav2Vec2 as w2v
from brain_solver import Filters, FilterType
from transformers.utils import logging
from tqdm import tqdm

# Suppress warnings if desired
import warnings

warnings.filterwarnings("ignore")
logging.set_verbosity(logging.CRITICAL)

# Setup for CUDA device selection
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
train_df: pd.DataFrame = hp.load_csv(config.data_train_csv)
test_df = pd.read_csv(config.data_test_csv)

if train_df is None:
    print("Failed to load the CSV file.")
    exit()
else:
    EEG_IDS = train_df.eeg_id.unique()
    TARGETS = train_df.columns[-6:]
    TARS = {"Seizure": 0, "LPD": 1, "GPD": 2, "LRDA": 3, "GRDA": 4, "Other": 5}
    TARS_INV = {x: y for y, x in TARS.items()}
    print("Train shape:", train_df.shape)

Train shape: (106800, 15)


In [6]:
spectrograms = hp.read_spectrograms(
    path=config.data_spectograms_test,
    data_path_train_on_brain_spectograms_dataset_specs=None,
    read_files=True,
)

# Continue with renaming for DataLoader
test_df = test_df.rename({"spectrogram_id": "spec_id"}, axis=1)

There are 1 spectrogram parquets
0 , 

In [7]:
# READ ALL EEG SPECTROGRAMS
DISPLAY = 1
EEG_IDS2 = test_df.eeg_id.unique()
all_eegs2 = {}

print("Converting Test EEG to Spectrograms...")
for i, eeg_id in enumerate(EEG_IDS2):
    all_eegs2[eeg_id] = hp.spectrogram_from_eeg(
        f"{config.data_eeg_test}{eeg_id}.parquet", False, config.use_wavelet
    )

Converting Test EEG to Spectrograms...


In [8]:
# INFER EFFICIENTNET ON TEST
preds = []
test_ds = EEGDataset(
    test_df, specs=spectrograms, eeg_specs=all_eegs2, targets=TARGETS, mode="test"
)
test_loader = DataLoader(test_ds, shuffle=False, batch_size=64, num_workers=3)

for i in range(5):
    print("#" * 25)
    print(f"### Testing Fold {i+1}")

    ckpt_file = (
        f"EffNet_version{config.VER}_fold{i+1}.pth"
        if config.trained_model_path is None
        else f"{config.trained_model_path}/EffNet_v{config.VER}_f{i}.ckpt"
    )
    model = torch.load(config.full_path + "trained-model-effnet-mlip9/" + ckpt_file)
    model = model.to(device).eval()
    fold_preds = []

    with torch.inference_mode():
        for test_batch in test_loader:
            test_batch = test_batch.to(device)
            pred = torch.softmax(model(test_batch), dim=1).cpu().numpy()
            fold_preds.append(pred)

            # Delete variables not needed to free up memory
            del test_batch, pred
            gc.collect()  # Manually collect garbage

            if device.type == "cuda":  # Optionally clear CUDA cache if using GPU
                torch.cuda.empty_cache()

        fold_preds = np.concatenate(fold_preds)

    preds.append(fold_preds)

    del model
    gc.collect()
    if device.type == "cuda":
        torch.cuda.empty_cache()

#########################
### Testing Fold 1
#########################
### Testing Fold 2
#########################
### Testing Fold 3
#########################
### Testing Fold 4
#########################
### Testing Fold 5


In [9]:
pred_model1 = np.mean(preds, axis=0)
print()
print("Test preds shape", pred_model1.shape)


Test preds shape (1, 6)


In [10]:
# Continue with renaming for DataLoader
test_df = test_df.rename({"spec_id": "spectrogram_id"}, axis=1)

In [11]:
# sub1 = pd.DataFrame({"eeg_id": test_df.eeg_id.values})
# sub1[TARGETS] = pred_model1[0]
# sub1.to_csv("submission_model1.csv", index=False)
# print("Submissionn shape", sub1.shape)
# sub1.head()

In [12]:
pred_model1

array([[0.02940221, 0.04890793, 0.00278009, 0.3094476 , 0.08677585,
        0.52268636]], dtype=float32)

In [13]:
# # SANITY CHECK TO CONFIRM PREDICTIONS SUM TO ONE
# sub1.iloc[:, -6:].sum(axis=1)

In [14]:
del spectrograms, all_eegs2
gc.collect()

0

# Model 2 - ResNet34d

In [15]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

import random
import warnings

warnings.filterwarnings("ignore")

In [16]:
image_transform = transforms.Resize((512, 512))

In [17]:
models = []
for i in range(config.num_folds):
    model = torch.load(f"{config.resnet34d}HMS_resnet_fold{i}.pth")
    models.append(model)
model = torch.load(
    f"{config.full_path}hms-baseline-resnet34d-512-512-training/HMS_resnet.pth"
)
models.append(model)

AttributeError: 'Config' object has no attribute 'num_folds'

In [ ]:
def seed_everything(seed):
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)


seed_everything(config.seed)

In [ ]:
submission = pd.read_csv(f"{config.competition_data_path}/sample_submission.csv")
submission = submission.merge(test_df, on="eeg_id", how="left")
submission["path"] = submission["spectrogram_id"].apply(
    lambda x: config.data_spectograms_test + str(x) + ".parquet"
)
submission.head()

In [ ]:
paths = submission["path"].values
pred_model2 = []
for path in paths:
    eps = 1e-6
    data = pd.read_parquet(path)

    data = data.fillna(-1).values[:, 1:].T
    data = data[:, 0:300]  # (400,300)
    data = np.clip(data, np.exp(-6), np.exp(10))
    data = np.log(data)
    data_mean = data.mean(axis=(0, 1))
    data_std = data.std(axis=(0, 1))
    data = (data - data_mean) / (data_std + eps)
    data_tensor = torch.unsqueeze(torch.Tensor(data), dim=0)
    data = image_transform(data_tensor)
    test_pred = []
    for model in models:
        model.eval()
        with torch.no_grad():
            pred = F.softmax(model(data.unsqueeze(0)))[0]
            pred = pred.detach().cpu().numpy()
        test_pred.append(pred)
    test_pred = np.array(test_pred).mean(axis=0)
    pred_model2.append(test_pred)
pred_model2 = np.array(pred_model2)
pred_model2

In [ ]:
sub2 = pd.read_csv(f"{config.competition_data_path}/sample_submission.csv")
labels = ["seizure", "lpd", "gpd", "lrda", "grda", "other"]
for i in range(len(labels)):
    sub2[f"{labels[i]}_vote"] = pred_model2[:, i]
sub2.head()
# sub2.to_csv("submission_model2.csv", index=False)

# Model 3 - ResNet34d, EfficientNetB0 and EfficientnetB1

In [ ]:
# Importing essential libraries
import gc
import os
import random
import warnings
import numpy as np
import pandas as pd
from IPython.display import display

# PyTorch for deep learning
import timm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# torchvision for image processing and augmentation
import torchvision.transforms as transforms

# Suppressing minor warnings to keep the output clean
warnings.filterwarnings("ignore", category=Warning)

# Reclaim memory no longer in use.
gc.collect()

In [ ]:
config.seed = 42
image_transform = transforms.Resize((512, 512))


# Set the seed for reproducibility across multiple libraries
def set_seed(seed):
    print(f"Setting seed non standard one: {seed}")
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)


set_seed(config.seed)

In [ ]:
# Load and store the trained models for each fold into a list
models = []

# Load ResNet34d
for i in range(config.num_folds):
    # Create the same model architecture as during training
    model_resnet = timm.create_model(
        "resnet34d", pretrained=False, num_classes=6, in_chans=1
    )

    # Load the trained weights from the corresponding file
    model_resnet.load_state_dict(
        torch.load(
            f"{config.train_resnet34d}resnet34d_fold{i}.pth",
            map_location=torch.device("cpu"),
        )
    )

    # Append the loaded model to the models list
    models.append(model_resnet)

# Reclaim memory no longer in use.
gc.collect()

# Load EfficientNetB0
for j in range(config.num_folds):
    # Create the same model architecture as during training
    model_effnet_b0 = timm.create_model(
        "efficientnet_b0", pretrained=False, num_classes=6, in_chans=1
    )

    # Load the trained weights from the corresponding file
    model_effnet_b0.load_state_dict(
        torch.load(
            f"{config.efficientnetb0}efficientnet_b0_fold{j}.pth",
            map_location=torch.device("cpu"),
        )
    )

    # Append the loaded model to the models list
    models.append(model_effnet_b0)

# Reclaim memory no longer in use.
gc.collect()

# Load EfficientNetB1
for k in range(config.num_folds):
    # Create the same model architecture as during training
    model_effnet_b1 = timm.create_model(
        "efficientnet_b1", pretrained=False, num_classes=6, in_chans=1
    )

    # Load the trained weights from the corresponding file
    model_effnet_b1.load_state_dict(
        torch.load(
            f"{config.efficientnetb1}efficientnet_b1_fold{k}.pth",
            map_location=torch.device("cpu"),
        )
    )

    # Append the loaded model to the models list
    models.append(model_effnet_b1)

# Reclaim memory no longer in use.
gc.collect()

In [ ]:
# Load test data and sample submission dataframe
test_df = pd.read_csv(config.data_test_csv)
submission = pd.read_csv(f"{config.competition_data_path}/sample_submission.csv")

# Merge the submission dataframe with the test data on EEG IDs
submission = submission.merge(test_df, on="eeg_id", how="left")

# Generate file paths for each spectrogram based on the EEG data in the submission dataframe
submission["path"] = submission["spectrogram_id"].apply(
    lambda x: f"{config.data_spectograms_test}{x}.parquet"
)

# Display the first few rows of the submission dataframe
display(submission.head())

# Reclaim memory no longer in use
gc.collect()

In [ ]:
# Define the weights for each model
weight_resnet34d = 0.26
weight_effnetb0 = 0.48
weight_effnetb1 = 0.26

# Get file paths for test spectrograms
paths = submission["path"].values
pred_model3 = []

# Generate predictions for each spectrogram using all models
for path in paths:
    eps = 1e-6
    # Read and preprocess spectrogram data
    data = pd.read_parquet(path)
    data = data.fillna(-1).values[:, 1:].T
    data = np.clip(data, np.exp(-6), np.exp(10))
    data = np.log(data)

    # Normalize the data
    data_mean = data.mean(axis=(0, 1))
    data_std = data.std(axis=(0, 1))
    data = (data - data_mean) / (data_std + eps)
    data_tensor = torch.unsqueeze(torch.Tensor(data), dim=0)
    data = image_transform(data_tensor)

    test_pred = []

    # Generate predictions using all models
    for model in models:
        model.eval()
        with torch.no_grad():
            pred = F.softmax(model(data.unsqueeze(0)))[0]
            pred = pred.detach().cpu().numpy()
        test_pred.append(pred)

    # Combine predictions from all models using weighted voting
    weighted_pred = (
        weight_resnet34d * np.mean(test_pred[: config.num_folds], axis=0)
        + weight_effnetb0
        * np.mean(test_pred[config.num_folds : 2 * config.num_folds], axis=0)
        + weight_effnetb1 * np.mean(test_pred[2 * config.num_folds :], axis=0)
    )

    pred_model3.append(weighted_pred)

# Convert the list of predictions to a NumPy array for further processing
pred_model3 = np.array(pred_model3)


# Reclaim memory no longer in use
gc.collect()

In [ ]:
# Model 3
# eeg_id_values = [3911565283]
# TARGETS = [
#     "seizure_vote",
#     "lpd_vote",
#     "gpd_vote",
#     "lrda_vote",
#     "grda_vote",
#     "other_vote",
# ]
# sub3 = pd.DataFrame({"eeg_id": eeg_id_values})
# sub3[TARGETS] = pred_model3
# print("Submission shape", sub3.shape)
# sub3.to_csv("submission_model3.csv", index=False)
# sub3.head()

In [ ]:
pred_model3

# Model 4 - Features+Head Ensemble 

In [ ]:
import os, random
import tensorflow as tf
import tensorflow
import tensorflow.keras.backend as K
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

LOAD_BACKBONE_FROM = config.efficientnetb_tf_keras
LOAD_MODELS_FROM = config.futures_head_starters_models
HMS_PATH = config.competition_data_path
VER = 50
DATA_TYPE = "KER"  # K|E|R|KE|KR|ER|KER
USE_PROCESSED = True  # Use processed downsampled Raw EEG
submission = True

# Setup for ensemble
ENSEMBLE = True
LBs = [
    0.41,
    0.39,
    0.41,
    0.37,
    0.39,
    0.38,
    0.36,
]  # K|E|R|KE|KR|ER|KER for weighted ensemble we use LBs of each model
VER_K = 43  # Kaggle's spectrogram model version
VER_E = 42  # EEG's spectrogram model version
VER_R = 37  # EEG's Raw wavenet model version, trained on single GPU
VER_KE = 47  # Kaggle's and EEG's spectrogram model version
VER_KR = 48  # Kaggle's spectrogram and Raw model version
VER_ER = 49  # EEG's spectrogram and Raw model version
VER_KER = 50  # EEG's, Kaggle's spectrograms and Raw model version

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

# USE SINGLE GPU, MULTIPLE GPUS
gpus = tf.config.list_physical_devices("GPU")
# WE USE MIXED PRECISION
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})
if len(gpus) > 1:
    strategy = tf.distribute.MirroredStrategy()
    print(f"Using {len(gpus)} GPUs")
else:
    strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    print(f"Using {len(gpus)} GPU")

TARGETS = [
    "seizure_vote",
    "lpd_vote",
    "gpd_vote",
    "lrda_vote",
    "grda_vote",
    "other_vote",
]

## DATA GENERATOR
This data generator outputs 512x512x3, the spectrogram and eeg images are concatenated all togother in a single image. For using data augmention you can set `augment = True` when creating the train data generator.

In [ ]:
import albumentations as albu
from scipy.signal import butter, lfilter
import librosa

FEATS2 = ["Fp1", "T3", "C3", "O1", "Fp2", "C4", "T4", "O2"]
FEAT2IDX = {x: y for x, y in zip(FEATS2, range(len(FEATS2)))}
FEATS = [
    ["Fp1", "F7", "T3", "T5", "O1"],
    ["Fp1", "F3", "C3", "P3", "O1"],
    ["Fp2", "F8", "T4", "T6", "O2"],
    ["Fp2", "F4", "C4", "P4", "O2"],
]


class DataGenerator:
    "Generates data for Keras"

    def __init__(
        self,
        data,
        specs=None,
        eeg_specs=None,
        raw_eegs=None,
        augment=False,
        mode="train",
        data_type=DATA_TYPE,
    ):
        self.data = data
        self.augment = augment
        self.mode = mode
        self.data_type = data_type
        self.specs = specs
        self.eeg_specs = eeg_specs
        self.raw_eegs = raw_eegs
        self.on_epoch_end()

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, index):
        X, y = self.data_generation(index)
        if self.augment:
            X = self.augmentation(X)
        return X, y

    def __call__(self):
        for i in range(self.__len__()):
            yield self.__getitem__(i)

            if i == self.__len__() - 1:
                self.on_epoch_end()

    def on_epoch_end(self):
        if self.mode == "train":
            self.data = self.data.sample(frac=1).reset_index(drop=True)

    def data_generation(self, index):
        if self.data_type == "KE":
            X, y = self.generate_all_specs(index)
        elif self.data_type == "E" or self.data_type == "K":
            X, y = self.generate_specs(index)
        elif self.data_type == "R":
            X, y = self.generate_raw(index)
        elif self.data_type in ["ER", "KR"]:
            X1, y = self.generate_specs(index)
            X2, y = self.generate_raw(index)
            X = (X1, X2)
        elif self.data_type in ["KER"]:
            X1, y = self.generate_all_specs(index)
            X2, y = self.generate_raw(index)
            X = (X1, X2)
        return X, y

    def generate_all_specs(self, index):
        X = np.zeros((512, 512, 3), dtype="float32")
        y = np.zeros((6,), dtype="float32")

        row = self.data.iloc[index]
        if self.mode == "test":
            offset = 0
        else:
            offset = int(row.offset / 2)

        eeg = self.eeg_specs[row.eeg_id]
        spec = self.specs[row.spec_id]

        imgs = [
            spec[offset : offset + 300, k * 100 : (k + 1) * 100].T for k in [0, 2, 1, 3]
        ]  # to match kaggle with eeg
        img = np.stack(imgs, axis=-1)
        # LOG TRANSFORM SPECTROGRAM
        img = np.clip(img, np.exp(-4), np.exp(8))
        img = np.log(img)

        # STANDARDIZE PER IMAGE
        img = np.nan_to_num(img, nan=0.0)

        mn = img.flatten().min()
        mx = img.flatten().max()
        ep = 1e-5
        img = 255 * (img - mn) / (mx - mn + ep)

        X[0_0 + 56 : 100 + 56, :256, 0] = img[:, 22:-22, 0]  # LL_k
        X[100 + 56 : 200 + 56, :256, 0] = img[:, 22:-22, 2]  # RL_k
        X[0_0 + 56 : 100 + 56, :256, 1] = img[:, 22:-22, 1]  # LP_k
        X[100 + 56 : 200 + 56, :256, 1] = img[:, 22:-22, 3]  # RP_k
        X[0_0 + 56 : 100 + 56, :256, 2] = img[:, 22:-22, 2]  # RL_k
        X[100 + 56 : 200 + 56, :256, 2] = img[:, 22:-22, 1]  # LP_k

        X[0_0 + 56 : 100 + 56, 256:, 0] = img[:, 22:-22, 0]  # LL_k
        X[100 + 56 : 200 + 56, 256:, 0] = img[:, 22:-22, 2]  # RL_k
        X[0_0 + 56 : 100 + 56, 256:, 1] = img[:, 22:-22, 1]  # LP_k
        X[100 + 56 : 200 + 56, 256:, 1] = img[:, 22:-22, 3]  # RP_K

        # EEG
        img = eeg
        mn = img.flatten().min()
        mx = img.flatten().max()
        ep = 1e-5
        img = 255 * (img - mn) / (mx - mn + ep)
        X[200 + 56 : 300 + 56, :256, 0] = img[:, 22:-22, 0]  # LL_e
        X[300 + 56 : 400 + 56, :256, 0] = img[:, 22:-22, 2]  # RL_e
        X[200 + 56 : 300 + 56, :256, 1] = img[:, 22:-22, 1]  # LP_e
        X[300 + 56 : 400 + 56, :256, 1] = img[:, 22:-22, 3]  # RP_e
        X[200 + 56 : 300 + 56, :256, 2] = img[:, 22:-22, 2]  # RL_e
        X[300 + 56 : 400 + 56, :256, 2] = img[:, 22:-22, 1]  # LP_e

        X[200 + 56 : 300 + 56, 256:, 0] = img[:, 22:-22, 0]  # LL_e
        X[300 + 56 : 400 + 56, 256:, 0] = img[:, 22:-22, 2]  # RL_e
        X[200 + 56 : 300 + 56, 256:, 1] = img[:, 22:-22, 1]  # LP_e
        X[300 + 56 : 400 + 56, 256:, 1] = img[:, 22:-22, 3]  # RP_e

        if self.mode != "test":
            y[:] = row[TARGETS]

        return X, y

    def generate_specs(self, index):
        X = np.zeros((512, 512, 3), dtype="float32")
        y = np.zeros((6,), dtype="float32")

        row = self.data.iloc[index]
        if self.mode == "test":
            offset = 0
        else:
            offset = int(row.offset / 2)

        if self.data_type in ["E", "ER"]:
            img = self.eeg_specs[row.eeg_id]
        elif self.data_type in ["K", "KR"]:
            spec = self.specs[row.spec_id]
            imgs = [
                spec[offset : offset + 300, k * 100 : (k + 1) * 100].T
                for k in [0, 2, 1, 3]
            ]  # to match kaggle with eeg
            img = np.stack(imgs, axis=-1)
            # LOG TRANSFORM SPECTROGRAM
            img = np.clip(img, np.exp(-4), np.exp(8))
            img = np.log(img)

            # STANDARDIZE PER IMAGE
            img = np.nan_to_num(img, nan=0.0)

        mn = img.flatten().min()
        mx = img.flatten().max()
        ep = 1e-5
        img = 255 * (img - mn) / (mx - mn + ep)

        X[0_0 + 56 : 100 + 56, :256, 0] = img[:, 22:-22, 0]
        X[100 + 56 : 200 + 56, :256, 0] = img[:, 22:-22, 2]
        X[0_0 + 56 : 100 + 56, :256, 1] = img[:, 22:-22, 1]
        X[100 + 56 : 200 + 56, :256, 1] = img[:, 22:-22, 3]
        X[0_0 + 56 : 100 + 56, :256, 2] = img[:, 22:-22, 2]
        X[100 + 56 : 200 + 56, :256, 2] = img[:, 22:-22, 1]

        X[0_0 + 56 : 100 + 56, 256:, 0] = img[:, 22:-22, 0]
        X[100 + 56 : 200 + 56, 256:, 0] = img[:, 22:-22, 1]
        X[0_0 + 56 : 100 + 56, 256:, 1] = img[:, 22:-22, 2]
        X[100 + 56 : 200 + 56, 256:, 1] = img[:, 22:-22, 3]

        X[200 + 56 : 300 + 56, :256, 0] = img[:, 22:-22, 0]
        X[300 + 56 : 400 + 56, :256, 0] = img[:, 22:-22, 1]
        X[200 + 56 : 300 + 56, :256, 1] = img[:, 22:-22, 2]
        X[300 + 56 : 400 + 56, :256, 1] = img[:, 22:-22, 3]
        X[200 + 56 : 300 + 56, :256, 2] = img[:, 22:-22, 3]
        X[300 + 56 : 400 + 56, :256, 2] = img[:, 22:-22, 2]

        X[200 + 56 : 300 + 56, 256:, 0] = img[:, 22:-22, 0]
        X[300 + 56 : 400 + 56, 256:, 0] = img[:, 22:-22, 2]
        X[200 + 56 : 300 + 56, 256:, 1] = img[:, 22:-22, 1]
        X[300 + 56 : 400 + 56, 256:, 1] = img[:, 22:-22, 3]

        if self.mode != "test":
            y[:] = row[TARGETS]

        return X, y

    def generate_raw(self, index):
        if USE_PROCESSED and self.mode != "test":
            X = np.zeros((2_000, 8), dtype="float32")
            y = np.zeros((6,), dtype="float32")
            row = self.data.iloc[index]
            X = self.raw_eegs[row.eeg_id]
            y[:] = row[TARGETS]
            return X, y

        X = np.zeros((10_000, 8), dtype="float32")
        y = np.zeros((6,), dtype="float32")

        row = self.data.iloc[index]
        eeg = self.raw_eegs[row.eeg_id]

        # FEATURE ENGINEER
        X[:, 0] = eeg[:, FEAT2IDX["Fp1"]] - eeg[:, FEAT2IDX["T3"]]
        X[:, 1] = eeg[:, FEAT2IDX["T3"]] - eeg[:, FEAT2IDX["O1"]]

        X[:, 2] = eeg[:, FEAT2IDX["Fp1"]] - eeg[:, FEAT2IDX["C3"]]
        X[:, 3] = eeg[:, FEAT2IDX["C3"]] - eeg[:, FEAT2IDX["O1"]]

        X[:, 4] = eeg[:, FEAT2IDX["Fp2"]] - eeg[:, FEAT2IDX["C4"]]
        X[:, 5] = eeg[:, FEAT2IDX["C4"]] - eeg[:, FEAT2IDX["O2"]]

        X[:, 6] = eeg[:, FEAT2IDX["Fp2"]] - eeg[:, FEAT2IDX["T4"]]
        X[:, 7] = eeg[:, FEAT2IDX["T4"]] - eeg[:, FEAT2IDX["O2"]]

        # STANDARDIZE
        X = np.clip(X, -1024, 1024)
        X = np.nan_to_num(X, nan=0) / 32.0

        # BUTTER LOW-PASS FILTER
        X = self.butter_lowpass_filter(X)
        # Downsample
        X = X[::5, :]

        if self.mode != "test":
            y[:] = row[TARGETS]

        return X, y

    def butter_lowpass_filter(self, data, cutoff_freq=20, sampling_rate=200, order=4):
        nyquist = 0.5 * sampling_rate
        normal_cutoff = cutoff_freq / nyquist
        b, a = butter(order, normal_cutoff, btype="low", analog=False)
        filtered_data = lfilter(b, a, data, axis=0)
        return filtered_data

    def resize(self, img, size):
        composition = albu.Compose([albu.Resize(size[0], size[1])])
        return composition(image=img)["image"]

    def augmentation(self, img):
        composition = albu.Compose([albu.HorizontalFlip(p=0.4)])
        return composition(image=img)["image"]


def spectrogram_from_eeg(parquet_path):

    # LOAD MIDDLE 50 SECONDS OF EEG SERIES
    eeg = pd.read_parquet(parquet_path)
    middle = (len(eeg) - 10_000) // 2
    eeg = eeg.iloc[middle : middle + 10_000]

    # VARIABLE TO HOLD SPECTROGRAM
    img = np.zeros((100, 300, 4), dtype="float32")

    for k in range(4):
        COLS = FEATS[k]

        for kk in range(4):
            # FILL NANS
            x1 = eeg[COLS[kk]].values
            x2 = eeg[COLS[kk + 1]].values
            m = np.nanmean(x1)
            if np.isnan(x1).mean() < 1:
                x1 = np.nan_to_num(x1, nan=m)
            else:
                x1[:] = 0
            m = np.nanmean(x2)
            if np.isnan(x2).mean() < 1:
                x2 = np.nan_to_num(x2, nan=m)
            else:
                x2[:] = 0

            # COMPUTE PAIR DIFFERENCES
            x = x1 - x2

            # RAW SPECTROGRAM
            mel_spec = librosa.feature.melspectrogram(
                y=x,
                sr=200,
                hop_length=len(x) // 300,
                n_fft=1024,
                n_mels=100,
                fmin=0,
                fmax=20,
                win_length=128,
            )

            # LOG TRANSFORM
            width = (mel_spec.shape[1] // 30) * 30
            mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max).astype(np.float32)[
                :, :width
            ]
            img[:, :, k] += mel_spec_db

        # AVERAGE THE 4 MONTAGE DIFFERENCES
        img[:, :, k] /= 4.0

    return img


def eeg_from_parquet(parquet_path):

    eeg = pd.read_parquet(parquet_path, columns=FEATS2)
    rows = len(eeg)
    offset = (rows - 10_000) // 2
    eeg = eeg.iloc[offset : offset + 10_000]
    data = np.zeros((10_000, len(FEATS2)))
    for j, col in enumerate(FEATS2):

        # FILL NAN
        x = eeg[col].values.astype("float32")
        m = np.nanmean(x)
        if np.isnan(x).mean() < 1:
            x = np.nan_to_num(x, nan=m)
        else:
            x[:] = 0

        data[:, j] = x

    return data

### MODEL AND UTILITY FUNCTIONS

In [ ]:
from tensorflow.keras.layers import (
    Input,
    Dense,
    Multiply,
    Add,
    Conv1D,
    Concatenate,
    LayerNormalization,
)


def build_model():
    K.clear_session()
    with strategy.scope():
        if DATA_TYPE in ["R"]:
            model = build_wave_model()
        elif DATA_TYPE in ["K", "E", "KE"]:
            model = build_spec_model()
        elif DATA_TYPE in ["KR", "ER", "KER"]:
            model = build_hybrid_model()
    return model


def build_spec_model(hybrid=False):
    inp = tf.keras.layers.Input((512, 512, 3))
    base_model = load_model(f"{LOAD_BACKBONE_FROM}")
    x = base_model(inp)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    if not hybrid:
        x = tf.keras.layers.Dense(6, activation="softmax", dtype="float32")(x)
    model = tf.keras.Model(inputs=inp, outputs=x)
    opt = tf.keras.optimizers.Adam(learning_rate=1e-3)
    loss = tf.keras.losses.KLDivergence()
    model.compile(loss=loss, optimizer=opt)
    return model


def wave_block(x, filters, kernel_size, n):
    dilation_rates = [2**i for i in range(n)]
    x = Conv1D(filters=filters, kernel_size=1, padding="same")(x)
    res_x = x
    for dilation_rate in dilation_rates:
        tanh_out = Conv1D(
            filters=filters,
            kernel_size=kernel_size,
            padding="same",
            activation="tanh",
            dilation_rate=dilation_rate,
        )(x)
        sigm_out = Conv1D(
            filters=filters,
            kernel_size=kernel_size,
            padding="same",
            activation="sigmoid",
            dilation_rate=dilation_rate,
        )(x)
        x = Multiply()([tanh_out, sigm_out])
        x = Conv1D(filters=filters, kernel_size=1, padding="same")(x)
        res_x = Add()([res_x, x])
    return res_x


def build_wave_model(hybrid=False):

    # INPUT
    inp = tf.keras.Input(shape=(2_000, 8))

    ############
    # FEATURE EXTRACTION SUB MODEL
    inp2 = tf.keras.Input(shape=(2_000, 1))
    x = wave_block(inp2, 8, 4, 6)
    x = wave_block(x, 16, 4, 6)
    x = wave_block(x, 32, 4, 6)
    x = wave_block(x, 64, 4, 6)
    model2 = tf.keras.Model(inputs=inp2, outputs=x)
    ###########

    # LEFT TEMPORAL CHAIN
    x1 = model2(inp[:, :, 0:1])
    x1 = tf.keras.layers.GlobalAveragePooling1D()(x1)
    x2 = model2(inp[:, :, 1:2])
    x2 = tf.keras.layers.GlobalAveragePooling1D()(x2)
    z1 = tf.keras.layers.Average()([x1, x2])

    # LEFT PARASAGITTAL CHAIN
    x1 = model2(inp[:, :, 2:3])
    x1 = tf.keras.layers.GlobalAveragePooling1D()(x1)
    x2 = model2(inp[:, :, 3:4])
    x2 = tf.keras.layers.GlobalAveragePooling1D()(x2)
    z2 = tf.keras.layers.Average()([x1, x2])

    # RIGHT PARASAGITTAL CHAIN
    x1 = model2(inp[:, :, 4:5])
    x1 = tf.keras.layers.GlobalAveragePooling1D()(x1)
    x2 = model2(inp[:, :, 5:6])
    x2 = tf.keras.layers.GlobalAveragePooling1D()(x2)
    z3 = tf.keras.layers.Average()([x1, x2])

    # RIGHT TEMPORAL CHAIN
    x1 = model2(inp[:, :, 6:7])
    x1 = tf.keras.layers.GlobalAveragePooling1D()(x1)
    x2 = model2(inp[:, :, 7:8])
    x2 = tf.keras.layers.GlobalAveragePooling1D()(x2)
    z4 = tf.keras.layers.Average()([x1, x2])

    # COMBINE CHAINS
    y = tf.keras.layers.Concatenate()([z1, z2, z3, z4])
    if not hybrid:
        y = tf.keras.layers.Dense(64, activation="relu")(y)
        y = tf.keras.layers.Dense(6, activation="softmax", dtype="float32")(y)

    # COMPILE MODEL
    model = tf.keras.Model(inputs=inp, outputs=y)
    opt = tf.keras.optimizers.Adam(learning_rate=1e-3)
    loss = tf.keras.losses.KLDivergence()
    model.compile(loss=loss, optimizer=opt)

    return model


def build_hybrid_model():
    model_spec = build_spec_model(True)
    model_wave = build_wave_model(True)
    inputs = [model_spec.input, model_wave.input]
    x = [model_spec.output, model_wave.output]
    x = tf.keras.layers.Concatenate()(x)
    x = tf.keras.layers.Dense(6, activation="softmax", dtype="float32")(x)

    # COMPILE MODEL
    model = tf.keras.Model(inputs=inputs, outputs=x)
    opt = tf.keras.optimizers.Adam(learning_rate=1e-3)
    loss = tf.keras.losses.KLDivergence()
    model.compile(loss=loss, optimizer=opt)

    return model


def score(y_true, y_pred):
    kl = tf.keras.metrics.KLDivergence()
    return kl(y_true, y_pred)


def plot_hist(hist):
    metrics = ["loss"]
    for i, metric in enumerate(metrics):
        plt.figure(figsize=(10, 4))
        plt.subplot(1, 2, i + 1)
        plt.plot(hist[metric])
        plt.plot(hist[f"val_{metric}"])
        plt.title(f"{metric}", size=12)
        plt.ylabel(f"{metric}", size=12)
        plt.xlabel("epoch", size=12)
        plt.legend(["train", "validation"], loc="upper left")
        plt.show()


def dataset(
    data,
    mode="train",
    batch_size=8,
    data_type=DATA_TYPE,
    augment=False,
    specs=None,
    eeg_specs=None,
    raw_eegs=None,
):

    BATCH_SIZE_PER_REPLICA = batch_size
    BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
    gen = DataGenerator(
        data,
        mode=mode,
        data_type=data_type,
        augment=augment,
        specs=specs,
        eeg_specs=eeg_specs,
        raw_eegs=raw_eegs,
    )
    if data_type in ["K", "E", "KE"]:
        inp = tf.TensorSpec(shape=(512, 512, 3), dtype=tf.float32)
    elif data_type in ["KR", "ER", "KER"]:
        inp = (
            tf.TensorSpec(shape=(512, 512, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(2000, 8), dtype=tf.float32),
        )
    elif data_type in ["R"]:
        inp = tf.TensorSpec(shape=(2000, 8), dtype=tf.float32)

    output_signature = (inp, tf.TensorSpec(shape=(6,), dtype=tf.float32))
    dataset = tf.data.Dataset.from_generator(
        generator=gen, output_signature=output_signature
    ).batch(BATCH_SIZE)
    return dataset


def reset_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)

## Infer Test and Create Submission CSV

Infer the test data and create a `config.output_path` file.


In [ ]:
if submission:
    test = pd.read_csv(config.data_test_csv)
    print("Test shape", test.shape)
    test.head()

In [ ]:
# READ ALL SPECTROGRAMS
if submission:
    files2 = os.listdir(config.data_spectograms_test)
    print(f"There are {len(files2)} test spectrogram parquets")

    spectrograms = {}
    for i, f in enumerate(files2):
        if i % 100 == 0:
            print(i, ", ", end="")
        tmp = pd.read_parquet(f"{config.data_spectograms_test}/{f}")
        name = int(f.split(".")[0])
        spectrograms[name] = tmp.iloc[:, 1:].values

    # RENAME FOR DATA GENERATOR
    test = test.rename({"spectrogram_id": "spec_id"}, axis=1)

In [ ]:
# READ ALL EEG SPECTROGRAMS
if submission:
    DISPLAY = 0
    EEG_IDS2 = test.eeg_id.unique()
    all_eegs2 = {}

    print("Converting Test EEG to Spectrograms...")
    print()
    for i, eeg_id in enumerate(EEG_IDS2):

        # CREATE SPECTROGRAM FROM EEG PARQUET
        img = spectrogram_from_eeg(f"{config.data_eeg_test}{eeg_id}.parquet")
        all_eegs2[eeg_id] = img

In [ ]:
# READ ALL RAW EEG SIGNALS
if submission:
    all_raw_eegs2 = {}
    EEG_IDS2 = test.eeg_id.unique()

    print("Processing Test EEG parquets...")
    print()
    for i, eeg_id in enumerate(EEG_IDS2):

        # SAVE EEG TO PYTHON DICTIONARY OF NUMPY ARRAYS
        data = eeg_from_parquet(f"{config.data_eeg_test}{eeg_id}.parquet")
        all_raw_eegs2[eeg_id] = data

In [ ]:
# Submission ON TEST with ensemble
if submission and ENSEMBLE:
    preds = []
    params = {"specs": spectrograms, "eeg_specs": all_eegs2, "raw_eegs": all_raw_eegs2}
    test_dataset_K = dataset(test, data_type="K", mode="test", **params)
    test_dataset_E = dataset(test, data_type="E", mode="test", **params)
    test_dataset_R = dataset(test, data_type="R", mode="test", **params)
    test_dataset_KE = dataset(test, data_type="KE", mode="test", **params)
    test_dataset_KR = dataset(test, data_type="KR", mode="test", **params)
    test_dataset_ER = dataset(test, data_type="ER", mode="test", **params)
    test_dataset_KER = dataset(test, data_type="KER", mode="test", **params)

    # LB SCORE WEIGHTS FOR EACH MODEL
    lbs = 1 - np.array(LBs)
    weights = lbs / lbs.sum()
    model_spec = build_spec_model()
    model_wave = build_wave_model()
    model_hybrid = build_hybrid_model()

    for i in range(5):
        print(f"Fold {i+1}")

        model_spec.load_weights(f"{LOAD_MODELS_FROM}/model_K_{VER_K}_{i}.weights.h5")
        pred_K = model_spec.predict(test_dataset_K, verbose=1)

        model_spec.load_weights(f"{LOAD_MODELS_FROM}/model_E_{VER_E}_{i}.weights.h5")
        pred_E = model_spec.predict(test_dataset_E, verbose=1)

        model_wave.load_weights(f"{LOAD_MODELS_FROM}/model_R_{VER_R}_{i}.weights.h5")
        pred_R = model_wave.predict(test_dataset_R, verbose=1)

        model_spec.load_weights(f"{LOAD_MODELS_FROM}/model_KE_{VER_KE}_{i}.weights.h5")
        pred_KE = model_spec.predict(test_dataset_KE, verbose=1)

        model_hybrid.load_weights(
            f"{LOAD_MODELS_FROM}/model_KR_{VER_KR}_{i}.weights.h5"
        )
        pred_KR = model_hybrid.predict(test_dataset_KR, verbose=1)

        model_hybrid.load_weights(
            f"{LOAD_MODELS_FROM}/model_ER_{VER_ER}_{i}.weights.h5"
        )
        pred_ER = model_hybrid.predict(test_dataset_ER, verbose=1)

        model_hybrid.load_weights(
            f"{LOAD_MODELS_FROM}/model_KER_{VER_KER}_{i}.weights.h5"
        )
        pred_KER = model_hybrid.predict(test_dataset_KER, verbose=1)

        pred = np.array([pred_K, pred_E, pred_R, pred_KE, pred_KR, pred_ER, pred_KER])
        pred = np.average(pred, axis=0, weights=weights)
        preds.append(pred)

    pred_model4 = np.mean(preds, axis=0)
    print("Test preds shape", pred_model4.shape)

In [ ]:
# FREE MEMORY
del model_spec, model_wave, model_hybrid
gc.collect()

In [ ]:
# if submission:
#     sub4 = pd.DataFrame({"eeg_id": test.eeg_id.values})
#     sub4[TARGETS] = pred_model4
#     print("Submissionn shape", sub4.shape)
#     print()
#     sub4.to_csv("submission_model4.csv", index=False)

In [ ]:
# # SANITY CHECK TO CONFIRM PREDICTIONS SUM TO ONE
# if submission:
#     print(sub4.iloc[:, -6:].sum(axis=1).to_string())

In [ ]:
pred_model4

# Model 5 - WaveNet

### [CV 0.68 | LB 0.46] DilatedInception WaveNet in PyTorch - Inference

#### Introduction
After joining this competition, I focus on validating how far raw EEG signals can go through many experiments. Finally, I find an simple architecture mixing the concept of **dilation** and **inception**, which can be seen as an extension of [Chris' version](https://www.kaggle.com/code/cdeotte/wavenet-starter-lb-0-52). And, I'm happy to announce that we can achieve CV 0.68 (below 0.7) and LB 0.46 with this architecture. Don't forget to upvote Chris' notebook!

#### About this Notebook
In this kernel, I run the inference process with 5-fold models (equally-weighted blending) and obtain LB 0.46. If you're also interested in training part, pleasse see [[LB 0.46] DilatedInception WaveNet - Training](https://www.kaggle.com/code/abaojiang/lb-0-46-dilatedinception-wavenet-training).

#### Acknowledgements
Special thanks to [@cdeotte](https://www.kaggle.com/cdeotte)'s sharing, [WaveNet Starter - [LB 0.52]](https://www.kaggle.com/code/cdeotte/wavenet-starter-lb-0-52).

<a id="toc"></a>
## Table of Contents
* [1. Load Data](#load_data)
* [2. Define Dataset](#dataset)
* [3. Create Test Loader](#test_loader)
* [4. Define Model Architecture](#model)
* [5. Load Models](#load_models)
* [6. Run Inference](#infer)
* [7. Submission](#sub)

#### Import Packages

In [ ]:
import gc
import os
from typing import Any, Dict, List, Optional, Tuple, Type, Union
import pickle
import warnings
from pathlib import Path
from tqdm.notebook import tqdm
warnings.simplefilter("ignore")

import numpy as np
import pandas as pd
import yaml
from scipy.signal import butter, lfilter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from torch.utils.data import Dataset, DataLoader

#### Define Data Paths and Configuration and Metadata

In [ ]:
DATA_PATH = Path(config.competition_data_path)

class CFG:
    exp_id = "0311-17-20-55"
    model_path = Path(f"{config.full_path}/dilated-wavenet/0311-17-20-55")
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    # == Data ==
    # Chris' 8 channels
    feats = [
        "Fp1", "T3", "C3", "O1",
        "Fp2", "C4", "T4", "O2"
    ]
    cast_eegs = True
    dataset = {
        "eeg": {
            "n_feats": 8,
            "apply_chris_magic_ch8": True,
            "normalize": True,
            "apply_butter_lowpass_filter": True,
            "apply_mu_law_encoding": False,
            "downsample": 5
        }
    }
    
    # == Data Loader ==
    batch_size = 32
    
    
N_CLASSES = 6
TGT_VOTE_COLS = [
    "seizure_vote", "lpd_vote", "gpd_vote", "lrda_vote",
    "grda_vote", "other_vote"
]
EEG_FREQ = 200  # Hz
EEG_WLEN = 50  # sec
EEG_PTS = int(EEG_FREQ * EEG_WLEN)

#### Load Data

Note test data will be replaced with **hidden test set** during rerun.

In [ ]:
def _get_eeg_window(file: Path) -> np.ndarray:
    """Return cropped EEG window.

    Default setting is to return the middle 50-sec window.

    Args:
        file: EEG file path
        test: if True, there's no need to truncate EEGs

    Returns:
        eeg_win: cropped EEG window 
    """
    eeg = pd.read_parquet(file, columns=CFG.feats)
    n_pts = len(eeg)
    offset = (n_pts - EEG_PTS) // 2
    eeg = eeg.iloc[offset:offset + EEG_PTS]
    
    eeg_win = np.zeros((EEG_PTS, len(CFG.feats)))
    for j, col in enumerate(CFG.feats):
        if CFG.cast_eegs:
            eeg_raw = eeg[col].values.astype("float32")
        else:
            eeg_raw = eeg[col].values 

        # Fill missing values
        mean = np.nanmean(eeg_raw)
        if np.isnan(eeg_raw).mean() < 1:
            eeg_raw = np.nan_to_num(eeg_raw, nan=mean)
        else: 
            # All missing
            eeg_raw[:] = 0
        eeg_win[:, j] = eeg_raw 
        
    return eeg_win 

In [ ]:
test = pd.read_csv(DATA_PATH / "test.csv")
print(f"Test data shape | {test.shape}")

In [ ]:
uniq_eeg_ids = test["eeg_id"].unique()
n_uniq_eeg_ids = len(uniq_eeg_ids)

all_eegs = {}
for i, eeg_id in tqdm(enumerate(uniq_eeg_ids), total=n_uniq_eeg_ids):
    eeg_win = _get_eeg_window(DATA_PATH / "test_eegs" / f"{eeg_id}.parquet")
    all_eegs[eeg_id] = eeg_win

print(f"Demo EEG shape | {list(all_eegs.values())[0].shape}")

#### 2. Define Dataset

In [ ]:
class _EEGTransformer(object):
    """Data transformer for raw EEG signals."""

    FEAT2CODE = {f: i for i, f in enumerate(CFG.feats)}

    def __init__(
        self,
        n_feats: int,
        apply_chris_magic_ch8: bool = True,
        normalize: bool = True,
        apply_butter_lowpass_filter: bool = True,
        apply_mu_law_encoding: bool = False,
        downsample: Optional[int] = None,
    ) -> None:
        self.n_feats = n_feats
        self.apply_chris_magic_ch8 = apply_chris_magic_ch8
        self.normalize = normalize
        self.apply_butter_lowpass_filter = apply_butter_lowpass_filter
        self.apply_mu_law_encoding = apply_mu_law_encoding
        self.downsample = downsample

    def transform(self, x: np.ndarray) -> np.ndarray:
        """Apply transformation on raw EEG signals.
        
        Args:
            x: raw EEG signals, with shape (L, C)

        Return:
            x_: transformed EEG signals
        """
        x_ = x.copy()
        if self.apply_chris_magic_ch8:
            x_ = self._apply_chris_magic_ch8(x_)

        if self.normalize:
            x_ = np.clip(x_, -1024, 1024)
            x_ = np.nan_to_num(x_, nan=0) / 32.0

        if self.apply_butter_lowpass_filter:
            x_ = self._butter_lowpass_filter(x_) 

        if self.apply_mu_law_encoding:
            x_ = self._quantize_data(x_, 1)

        if self.downsample is not None:
            x_ = x_[::self.downsample, :]

        return x_

    def _apply_chris_magic_ch8(self, x: np.ndarray) -> np.ndarray:
        """Generate features based on Chris' magic formula.""" 
        x_tmp = np.zeros((EEG_PTS, self.n_feats), dtype="float32")

        # Generate features
        x_tmp[:, 0] = x[:, self.FEAT2CODE["Fp1"]] - x[:, self.FEAT2CODE["T3"]]
        x_tmp[:, 1] = x[:, self.FEAT2CODE["T3"]] - x[:, self.FEAT2CODE["O1"]]
        
        x_tmp[:, 2] = x[:, self.FEAT2CODE["Fp1"]] - x[:, self.FEAT2CODE["C3"]]
        x_tmp[:, 3] = x[:, self.FEAT2CODE["C3"]] - x[:, self.FEAT2CODE["O1"]]
        
        x_tmp[:, 4] = x[:, self.FEAT2CODE["Fp2"]] - x[:, self.FEAT2CODE["C4"]]
        x_tmp[:, 5] = x[:, self.FEAT2CODE["C4"]] - x[:, self.FEAT2CODE["O2"]]
        
        x_tmp[:, 6] = x[:, self.FEAT2CODE["Fp2"]] - x[:, self.FEAT2CODE["T4"]]
        x_tmp[:, 7] = x[:, self.FEAT2CODE["T4"]] - x[:, self.FEAT2CODE["O2"]]

        return x_tmp

    def _butter_lowpass_filter(self, data, cutoff_freq=20, sampling_rate=200, order=4):
        nyquist = 0.5 * sampling_rate
        normal_cutoff = cutoff_freq / nyquist
        b, a = butter(order, normal_cutoff, btype="low", analog=False)
        filtered_data = lfilter(b, a, data, axis=0)

        return filtered_data
                
    def _quantize_data(self, data, classes):
        mu_x = self._mu_law_encoding(data, classes)
        
        return mu_x

    def _mu_law_encoding(self, data, mu):
        mu_x = np.sign(data) * np.log(1 + mu * np.abs(data)) / np.log(mu + 1)

        return mu_x

In [ ]:
class EEGDataset(Dataset):
    """Dataset for pure raw EEG signals.

    Args:
        data: processed data
        split: data split

    Attributes:
        _n_samples: number of samples
        _infer: if True, the dataset is constructed for inference
            *Note: Ground truth is not provided.
    """

    def __init__(
        self,
        data: Dict[str,  Any],
        split: str,
        **dataset_cfg: Any,
    ) -> None:
        self.metadata = data["meta"]
        self.all_eegs = data["eeg"]
        self.dataset_cfg = dataset_cfg

        # Raw EEG data transformer
        self.eeg_params = dataset_cfg["eeg"]
        self.eeg_trafo = _EEGTransformer(**self.eeg_params)

        self._set_n_samples()
        self._infer = True if split == "test" else False

        self._stream_X = True if self.all_eegs is None else False
        self._X, self._y = self._transform()

    def _set_n_samples(self) -> None:
        assert len(self.metadata) == self.metadata["eeg_id"].nunique()
        self._n_samples = len(self.metadata)

    def _transform(self) -> Tuple[Optional[np.ndarray], np.ndarray]:
        """Transform feature and target matrices."""
        if self.eeg_params["downsample"] is not None:
            eeg_len = int(EEG_PTS / self.eeg_params["downsample"])
        else:
            eeg_len = int(EEG_PTS)
        if not self._stream_X:
            X = np.zeros((self._n_samples, eeg_len, self.eeg_params["n_feats"]), dtype="float32")
        else:
            X = None
        y = np.zeros((self._n_samples, N_CLASSES), dtype="float32") if not self._infer else None

        for i, row in tqdm(self.metadata.iterrows(), total=len(self.metadata)):
            # Process raw EEG signals
            if not self._stream_X:
                # Retrieve raw EEG signals
                eeg = self.all_eegs[row["eeg_id"]]

                # Apply EEG transformer
                x = self.eeg_trafo.transform(eeg)

                X[i] = x

            if not self._infer:
                y[i] = row[TGT_VOTE_COLS] 

        return X, y

    def __len__(self) -> int:
        return self._n_samples

    def __getitem__(self, idx: int) -> Dict[str, Tensor]:
        if self._X is None:
            # Load data here...
#             x = np.load(...)
#             x = self.eeg_trafo.transform(x)
            pass
        else:
            x = self._X[idx, ...]
        data_sample = {"x": torch.tensor(x, dtype=torch.float32)}
        if not self._infer:
            data_sample["y"] = torch.tensor(self._y[idx, :], dtype=torch.float32)

        return data_sample

<a id="test_loader"></a>
## 3. Create Test Loader
[**<span style="color:#FEF1FE; background-color:#535d70;border-radius: 5px; padding: 2px">Go to Table of Content</span>**](#toc)

In [ ]:
test_data = {"meta": test, "eeg": all_eegs}
test_loader = DataLoader(
    EEGDataset(test_data, "test", **CFG.dataset),
    batch_size=CFG.batch_size,
    shuffle=False,
    num_workers=0
)
print(f"There are {len(test_loader.dataset)} test samples to infer.")

#### 4. Define Model Architecture

[![Screenshot-2024-02-19-at-1-11-40-PM.png](https://i.postimg.cc/MKp8xVVV/Screenshot-2024-02-19-at-1-11-40-PM.png)](https://postimg.cc/7bdRnCBZ)

In [ ]:
class _WaveBlock(nn.Module):
    """WaveNet block.

    Args:
        kernel_size: kernel size, pass a list of kernel sizes for
            inception
    """

    def __init__(
        self,
        n_layers: int, 
        in_dim: int,
        h_dim: int,
        kernel_size: Union[int, List[int]],
        conv_module: Optional[Type[nn.Module]] = None,
    ) -> None:
        super().__init__()

        self.n_layers = n_layers
        self.dilation_rates = [2**l for l in range(n_layers)]

        self.in_conv = nn.Conv2d(in_dim, h_dim, kernel_size=(1, 1)) 
        self.gated_tcns = nn.ModuleList()
        self.skip_convs = nn.ModuleList()
        for layer in range(n_layers):
            c_in, c_out = h_dim, h_dim
            self.gated_tcns.append(
                _GatedTCN(
                    in_dim=c_in,
                    h_dim=c_out,
                    kernel_size=kernel_size,
                    dilation_factor=self.dilation_rates[layer],
                    conv_module=conv_module,
                )
            )
            self.skip_convs.append(nn.Conv2d(h_dim, h_dim, kernel_size=(1, 1)))

        # Initialize parameters
        nn.init.xavier_uniform_(self.in_conv.weight, gain=nn.init.calculate_gain("relu"))
        nn.init.zeros_(self.in_conv.bias)
        for i in range(len(self.skip_convs)):
            nn.init.xavier_uniform_(self.skip_convs[i].weight, gain=nn.init.calculate_gain("relu"))
            nn.init.zeros_(self.skip_convs[i].bias)

    def forward(self, x: Tensor) -> Tensor:
        """Forward pass.
        
        Shape:
            x: (B, C, N, L), where C denotes in_dim
            x_skip: (B, C', N, L), where C' denotes h_dim
        """
        # Input convolution
        x = self.in_conv(x)

        x_skip = x
        for layer in range(self.n_layers):
            x = self.gated_tcns[layer](x)
            x = self.skip_convs[layer](x)

            # Skip-connection
            x_skip = x_skip + x 

        return x_skip


class _GatedTCN(nn.Module):
    """Gated temporal convolution layer.

    Parameters:
        conv_module: customized convolution module
    """

    def __init__(
        self,
        in_dim: int,
        h_dim: int,
        kernel_size: Union[int, List[int]],
        dilation_factor: int,
        dropout: Optional[float] = None,
        conv_module: Optional[Type[nn.Module]] = None,
    ) -> None:
        super().__init__()

        # Model blocks
        if conv_module is None:
            self.filt = nn.Conv2d(
                in_channels=in_dim, out_channels=h_dim, kernel_size=(1, kernel_size), dilation=dilation_factor
            )
            self.gate = nn.Conv2d(
                in_channels=in_dim, out_channels=h_dim, kernel_size=(1, kernel_size), dilation=dilation_factor
            )
        else:
            self.filt = conv_module(
                in_channels=in_dim, out_channels=h_dim, kernel_size=kernel_size, dilation=dilation_factor
            )
            self.gate = conv_module(
                in_channels=in_dim, out_channels=h_dim, kernel_size=kernel_size, dilation=dilation_factor
            )

        if dropout is not None:
            self.dropout = nn.Dropout(dropout)
        else:
            self.dropout = None

    def forward(self, x: Tensor) -> Tensor:
        """Forward pass.

        Parameters:
            x: input sequence

        Return:
            h: output sequence

        Shape:
            x: (B, C, N, L), where L denotes the input sequence length
            h: (B, h_dim, N, L')
        """
        x_filt = F.tanh(self.filt(x))
        x_gate = F.sigmoid(self.gate(x))
        h = x_filt * x_gate
        if self.dropout is not None:
            h = self.dropout(h)

        return h


class _DilatedInception(nn.Module):
    """Dilated inception layer.

    Note that `out_channels` will be split across #kernels.
    """

    def __init__(
        self, 
        in_channels: int, 
        out_channels: int, 
        kernel_size: List[int], 
        dilation: int
    ) -> None:
        super().__init__()

        # Network parameters
        n_kernels = len(kernel_size)
        assert out_channels % n_kernels == 0, "`out_channels` must be divisible by #kernels."
        h_dim = out_channels // n_kernels

        # Model blocks
        self.convs = nn.ModuleList()
        for k in kernel_size:
            self.convs.append(
                nn.Conv2d(
                    in_channels=in_channels, 
                    out_channels=h_dim, 
                    kernel_size=(1, k),
                    padding="same",
                    dilation=dilation),
            )

    def forward(self, x: Tensor) -> Tensor:
        """Forward pass.

        Parameters:
            x: input sequence

        Return:
            h: output sequence

        Shape:
            x: (B, C, N, L), where C = in_channels
            h: (B, C', N, L'), where C' = out_channels
        """
        x_convs = []
        for conv in self.convs:
            x_conv = conv(x)
            x_convs.append(x_conv)
        h = torch.cat(x_convs, dim=1)

        return h

In [ ]:
class DilatedInceptionWaveNet(nn.Module):
    """WaveNet architecture with dilated inception conv."""

    def __init__(self,) -> None:
        super().__init__()

        kernel_size = [2, 3, 6, 7]

        # Model blocks 
        self.wave_module = nn.Sequential(
            _WaveBlock(12, 1, 16, kernel_size, _DilatedInception),
            _WaveBlock(8, 16, 32, kernel_size, _DilatedInception),
            _WaveBlock(4, 32, 64, kernel_size, _DilatedInception),
            _WaveBlock(1, 64, 64, kernel_size, _DilatedInception),
        )
        self.output = nn.Sequential(
            nn.Linear(64 * 4, 64),
            nn.ReLU(),
            nn.Linear(64, N_CLASSES)
        ) 

    def forward(self, inputs: Dict[str, Tensor]) -> Tensor:
        """Forward pass.

        Shape:
            x: (B, L, C)
        """
        x = inputs["x"]
        bs, length, in_dim = x.shape
        x = x.transpose(1, 2).unsqueeze(dim=2)  # (B, C, N, L), N is redundant

        x_ll_1 = self.wave_module(x[:, 0:1, :])
        x_ll_2 = self.wave_module(x[:, 1:2, :])
        x_ll = (F.adaptive_avg_pool2d(x_ll_1, (1, 1)) + F.adaptive_avg_pool2d(x_ll_2, (1, 1))) / 2

        x_rl_1 = self.wave_module(x[:, 2:3, :])
        x_rl_2 = self.wave_module(x[:, 3:4, :])
        x_rl = (F.adaptive_avg_pool2d(x_rl_1, (1, 1)) + F.adaptive_avg_pool2d(x_rl_2, (1, 1))) / 2

        x_lp_1 = self.wave_module(x[:, 4:5, :])
        x_lp_2 = self.wave_module(x[:, 5:6, :])
        x_lp = (F.adaptive_avg_pool2d(x_lp_1, (1, 1)) + F.adaptive_avg_pool2d(x_lp_2, (1, 1))) / 2

        x_rp_1 = self.wave_module(x[:, 6:7, :])
        x_rp_2 = self.wave_module(x[:, 7:8, :])
        x_rp = (F.adaptive_avg_pool2d(x_rp_1, (1, 1)) + F.adaptive_avg_pool2d(x_rp_2, (1, 1))) / 2

        x = torch.cat([x_ll, x_rl, x_lp, x_rp], axis=1).reshape(bs, -1)
        output = self.output(x)

        return output

#### 5. Load Models

In [ ]:
models = []
for fold, file in enumerate(sorted(CFG.model_path.glob("./*.pth"))):
    print(f"Load model from {file}...")
    fold_model = DilatedInceptionWaveNet()
    fold_model.load_state_dict(torch.load(file, map_location=CFG.device))
    fold_model = fold_model.to(CFG.device)
    models.append(fold_model)

#### 6. Run Inference

In [ ]:
@torch.no_grad()
def _infer(inputs: Dict[str, Tensor], models: List[nn.Module]) -> Tensor:
    n_models = len(models)

    for i, model in enumerate(models):
        model.eval()
        y_pred_fold = F.softmax(model(inputs)) / n_models    # (B, N_CLASSES)
        
        if i == 0:
            y_pred = y_pred_fold
        else:
            y_pred += y_pred_fold
        
    return y_pred

In [ ]:
y_preds = []
for i, batch_data in enumerate(test_loader):
    batch_data["x"] = batch_data["x"].to(CFG.device)
    y_pred = _infer(batch_data, models)
    y_preds.append(y_pred.detach().cpu().numpy())
y_preds = np.vstack(y_preds)
print(f"Sum of row 0 in y_preds {np.sum(y_preds[0, :])}.")

#### 7. Submission

In [ ]:
pred_model5 = y_pred

In [ ]:
# if submission:
#     sub5 = pd.DataFrame({"eeg_id": test.eeg_id.values})
#     sub5[TARGETS] = pred_model5
#     print("Submissionn shape", sub5.shape)
#     print()
#     print(sub5.head().to_string())
# #     sub5.to_csv("submission_model5.csv", index=False)
#     sub5.head()

In [ ]:
# # SANITY CHECK TO CONFIRM PREDICTIONS SUM TO ONE
# if submission:
#     print(sub5.iloc[:, -6:].sum(axis=1).to_string())

# Model 6 - CatBoost Starter

## CatBoost Starter for Brain Comp
This is a CatBoost starter notebook for Kaggle's brain comp. We use only spectrogram features. (The model does not use eeg features yet). We can improve the CV and LB score by engineering more (spectrogram and/or eeg) features and we can tune the CatBoost model (and/or use other ML DL models). Discussion about this starter is [here][2].

In this notebook, we also compare five CV scores. Kaggle's sample submission uses equal predictions of 1/6 for all targets and achieves CV 1.46, LB 1.09. The best public notebook (on Jan 12th) [here][1] uses train means and achieves CV 1.26 LB 0.97. Our CatBoost model version 1 achieves CV 1.01 LB 0.81. Our CatBoost model version 2 achieves CV 0.82 LB 0.67. Then version 3 adds features from **EEG spectrograms** and achieves CV 0.74, wow! Let's see what LB is...

### Exciting UPDATE!
Version 3 of this notebook trains using **both** Kaggle spectrograms and my new **EEG spectrograms** from my Kaggle dataset [here][3] (which were created from my spectrogram starter [here][4]). We boost the CV score and (most likely) LB score by almost `+0.10`, wow! 

#### Version Notes
* Version 1 - Uses spectrogram features from 10 minute window `means`. Achieves CV 1.01, LB 0.81
* Version 2 - Uses spectrogram features from 10 minute and 20 second `means` and `mins`. Achieves CV 0.82, LB 0.67
* Version 3 - Uses Kaggle spectrogrms **plus EEG spectrograms**. Achieves 0.74, LB to be determined...

[1]: https://www.kaggle.com/code/seshurajup/eda-train-csv
[2]: https://www.kaggle.com/competitions/hms-harmful-brain-activity-classification/discussion/467576
[3]: https://www.kaggle.com/datasets/cdeotte/brain-eeg-spectrograms
[4]: https://www.kaggle.com/code/cdeotte/how-to-make-spectrogram-from-eeg

### Load Libraries

In [ ]:
import os, gc

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
import pandas as pd, numpy as np
import matplotlib.pyplot as plt

VER = 3

### Load Train Data

In [ ]:
df = pd.read_csv(config.data_train_csv)
TARGETS = df.columns[-6:]
print("Train shape:", df.shape)
print("Targets", list(TARGETS))
df.head()

### Create Non-Overlapping Eeg Id Train Data
The competition data description says that test data does not have multiple crops from the same `eeg_id`. Therefore we will train and validate using only 1 crop per `eeg_id`. There is a discussion about this [here][1].

[1]: https://www.kaggle.com/competitions/hms-harmful-brain-activity-classification/discussion/467021

In [ ]:
train = hp.preprocess_eeg_data(train_df, TARGETS)

### Infer Test and Create Submission CSV
Below we use our 5 CatBoost fold models to infer the test data and create a `submission.csv` file.

In [ ]:
import pywt, librosa

USE_WAVELET = None

NAMES = ["LL", "LP", "RP", "RR"]

FEATS = [
    ["Fp1", "F7", "T3", "T5", "O1"],
    ["Fp1", "F3", "C3", "P3", "O1"],
    ["Fp2", "F8", "T4", "T6", "O2"],
    ["Fp2", "F4", "C4", "P4", "O2"],
]


# DENOISE FUNCTION
def maddest(d, axis=None):
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)


def denoise(x, wavelet="haar", level=1):
    coeff = pywt.wavedec(x, wavelet, mode="per")
    sigma = (1 / 0.6745) * maddest(coeff[-level])

    uthresh = sigma * np.sqrt(2 * np.log(len(x)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode="hard") for i in coeff[1:])

    ret = pywt.waverec(coeff, wavelet, mode="per")

    return ret


def spectrogram_from_eeg(parquet_path, display=False):

    # LOAD MIDDLE 50 SECONDS OF EEG SERIES
    eeg = pd.read_parquet(parquet_path)
    middle = (len(eeg) - 10_000) // 2
    eeg = eeg.iloc[middle : middle + 10_000]

    # VARIABLE TO HOLD SPECTROGRAM
    img = np.zeros((128, 256, 4), dtype="float32")

    if display:
        plt.figure(figsize=(10, 7))
    signals = []
    for k in range(4):
        COLS = FEATS[k]

        for kk in range(4):

            # COMPUTE PAIR DIFFERENCES
            x = eeg[COLS[kk]].values - eeg[COLS[kk + 1]].values

            # FILL NANS
            m = np.nanmean(x)
            if np.isnan(x).mean() < 1:
                x = np.nan_to_num(x, nan=m)
            else:
                x[:] = 0

            # DENOISE
            if USE_WAVELET:
                x = denoise(x, wavelet=USE_WAVELET)
            signals.append(x)

            # RAW SPECTROGRAM
            mel_spec = librosa.feature.melspectrogram(
                y=x,
                sr=200,
                hop_length=len(x) // 256,
                n_fft=1024,
                n_mels=128,
                fmin=0,
                fmax=20,
                win_length=128,
            )

            # LOG TRANSFORM
            width = (mel_spec.shape[1] // 32) * 32
            mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max).astype(np.float32)[
                :, :width
            ]

            # STANDARDIZE TO -1 TO 1
            mel_spec_db = (mel_spec_db + 40) / 40
            img[:, :, k] += mel_spec_db

        # AVERAGE THE 4 MONTAGE DIFFERENCES
        img[:, :, k] /= 4.0

        if display:
            plt.subplot(2, 2, k + 1)
            plt.imshow(img[:, :, k], aspect="auto", origin="lower")
            plt.title(f"EEG {eeg_id} - Spectrogram {NAMES[k]}")

    if display:
        plt.show()
        plt.figure(figsize=(10, 5))
        offset = 0
        for k in range(4):
            if k > 0:
                offset -= signals[3 - k].min()
            plt.plot(range(10_000), signals[k] + offset, label=NAMES[3 - k])
            offset += signals[3 - k].max()
        plt.legend()
        plt.title(f"EEG {eeg_id} Signals")
        plt.show()
        print()
        print("#" * 25)
        print()

    return img

In [ ]:
# CREATE ALL EEG SPECTROGRAMS
DISPLAY = 0
EEG_IDS2 = test.eeg_id.unique()
all_eegs2 = {}

print("Converting Test EEG to Spectrograms...")
print()
for i, eeg_id in enumerate(EEG_IDS2):

    # CREATE SPECTROGRAM FROM EEG PARQUET
    all_eegs2[eeg_id] = spectrogram_from_eeg(
        f"{config.data_eeg_test}{eeg_id}.parquet", False
    )

In [ ]:
# FEATURE NAMES
SPEC_COLS = pd.read_parquet(f"{config.data_spectograms}1000086677.parquet").columns[1:]
FEATURES = [f"{c}_mean_10m" for c in SPEC_COLS]
FEATURES += [f"{c}_min_10m" for c in SPEC_COLS]
FEATURES += [f"{c}_mean_20s" for c in SPEC_COLS]
FEATURES += [f"{c}_min_20s" for c in SPEC_COLS]
FEATURES += [f"eeg_mean_f{x}_10s" for x in range(512)]
FEATURES += [f"eeg_min_f{x}_10s" for x in range(512)]
FEATURES += [f"eeg_max_f{x}_10s" for x in range(512)]
FEATURES += [f"eeg_std_f{x}_10s" for x in range(512)]
print(f"We are creating {len(FEATURES)} features for {len(train)} rows... ", end="")

In [ ]:
# FEATURE ENGINEER TEST
data = np.zeros((len(test), len(FEATURES)))

for k in range(len(test)):
    row = test.iloc[k]
    s = int(row.spec_id)
    spec = pd.read_parquet(f"{config.data_spectograms_test}{s}.parquet")

    # 10 MINUTE WINDOW FEATURES
    x = np.nanmean(spec.iloc[:, 1:].values, axis=0)
    data[k, :400] = x
    x = np.nanmin(spec.iloc[:, 1:].values, axis=0)
    data[k, 400:800] = x

    # 20 SECOND WINDOW FEATURES
    x = np.nanmean(spec.iloc[145:155, 1:].values, axis=0)
    data[k, 800:1200] = x
    x = np.nanmin(spec.iloc[145:155, 1:].values, axis=0)
    data[k, 1200:1600] = x

    # RESHAPE EEG SPECTROGRAMS 128x256x4 => 512x256
    eeg_spec = np.zeros((512, 256), dtype="float32")
    xx = all_eegs2[row.eeg_id]
    for j in range(4):
        eeg_spec[128 * j : 128 * (j + 1),] = xx[:, :, j]

    # 10 SECOND WINDOW FROM EEG SPECTROGRAMS
    x = np.nanmean(eeg_spec.T[100:-100, :], axis=0)
    data[k, 1600:2112] = x
    x = np.nanmin(eeg_spec.T[100:-100, :], axis=0)
    data[k, 2112:2624] = x
    x = np.nanmax(eeg_spec.T[100:-100, :], axis=0)
    data[k, 2624:3136] = x
    x = np.nanstd(eeg_spec.T[100:-100, :], axis=0)
    data[k, 3136:3648] = x

test[FEATURES] = data
print("New test shape", test.shape)

In [ ]:
import catboost as cat
from catboost import CatBoostClassifier, Pool

In [ ]:
# INFER CATBOOST ON TEST
preds = []

for i in range(5):
    print(i, ", ", end="")
    model = CatBoostClassifier(task_type="GPU")
    model.load_model(
        f"{config.full_path}catboost-model/catboost_model/CAT_v{VER}_f{i}.cat"
    )

    test_pool = Pool(data=test[FEATURES])

    pred = model.predict_proba(test_pool)
    preds.append(pred)
pred_model6 = np.mean(preds, axis=0)
print()
print("Test preds shape", pred.shape)

In [ ]:
# FREE MEMORY
del data
gc.collect()

In [ ]:
sub6 = pd.DataFrame({"eeg_id": test.eeg_id.values})
sub6[TARGETS] = pred_model6
sub6.to_csv("submission.csv", index=False)
print("Submissionn shape", sub6.shape)
sub6.head()

In [ ]:
# SANITY CHECK TO CONFIRM PREDICTIONS SUM TO ONE
sub6.iloc[:, -6:].sum(axis=1)

# Submission

In [ ]:
import numpy as np
# Scores for each model, where lower is better
scores = np.array([0.43, 0.45, 0.41, 0.34, 0.46, 0.60])

inverted_scores = 1 / scores
transformed_scores = inverted_scores ** 8
weights = transformed_scores / transformed_scores.sum()
weights

In [ ]:
use_computed_weights = True
manual_weights = [0.05, 0.05, 0.25, 0.65, 0.00, 0.05]
labels = ["seizure", "lpd", "gpd", "lrda", "grda", "other"]
submission = pd.read_csv(f"{config.competition_data_path}/sample_submission.csv")

In [ ]:
for i in range(len(labels)):
    if use_computed_weights:
        # Use the computed weights
        submission[f"{labels[i]}_vote"] = (
            pred_model1[:, i] * weights[0] +
            pred_model2[:, i] * weights[1] +
            pred_model3[:, i] * weights[2] +
            pred_model4[:, i] * weights[3] +
            pred_model5[:, i] * weights[4] +
            pred_model6[:, i] * weights[5]
        )
    else:
        # Use the manually set weights
        submission[f"{labels[i]}_vote"] = (
            pred_model1[:, i] * manual_weights[0] +
            pred_model2[:, i] * manual_weights[1] +
            pred_model3[:, i] * manual_weights[2] +
            pred_model4[:, i] * manual_weights[3] +
            pred_model5[:, i] * manual_weights[4] +
            pred_model6[:, i] * manual_weights[5]
        )
submission.to_csv(f"{config.output_path}submission.csv", index=None)
display(submission.head())

In [ ]:
# # SANITY CHECK TO CONFIRM PREDICTIONS SUM TO ONE
# submission.iloc[:, -6:].sum(axis=1)

In [ ]:
# delete this code
# tempz=pd.read_csv(config.output_path)

# tempz.iloc[0, 1] = 0.098192
# tempz.iloc[0, 2] = 0.058772
# tempz.iloc[0, 3] = 0.000671
# tempz.iloc[0, 4] = 0.395016
# tempz.iloc[0, 5] = 0.023945
# tempz.iloc[0, 6] = 0.423405

# tempz.to_csv("/kaggle/working/submission.csv", index=False)
# display(tempz.head())

In [ ]:
import pandas as pd


# Read the CSV file
finalrounding = pd.read_csv(f"{config.output_path}submission.csv")

seizure_vote_value = finalrounding.iloc[0, 1]

lpd_vote_value = finalrounding.iloc[0, 2]

gpd_vote_value = finalrounding.iloc[0, 3]

lrda_vote_value = finalrounding.iloc[0, 4]

grda_vote_value = finalrounding.iloc[0, 5]

other_vote_value = finalrounding.iloc[0, 6]

buffer_value = 0.0

if seizure_vote_value < 0.06:
    buffer_value = buffer_value + seizure_vote_value
    finalrounding.iloc[0, 1] = 0
    seizure_vote_value = 0

if lpd_vote_value < 0.06:
    buffer_value = buffer_value + lpd_vote_value
    finalrounding.iloc[0, 2] = 0
    lpd_vote_value = 0

if gpd_vote_value < 0.06:
    buffer_value = buffer_value + gpd_vote_value
    finalrounding.iloc[0, 3] = 0
    gpd_vote_value = 0

if lrda_vote_value < 0.06:
    buffer_value = buffer_value + lrda_vote_value
    finalrounding.iloc[0, 4] = 0
    lrda_vote_value = 0

if grda_vote_value < 0.06:
    buffer_value = buffer_value + grda_vote_value
    finalrounding.iloc[0, 5] = 0
    grda_vote_value = 0

if other_vote_value < 0.06:
    buffer_value = buffer_value + other_vote_value
    finalrounding.iloc[0, 6] = 0
    other_vote_value = 0

vote_dict = {
    "var1": seizure_vote_value,
    "var2": lpd_vote_value,
    "var3": gpd_vote_value,
    "var4": lrda_vote_value,
    "var5": grda_vote_value,
}

biggest_var_name = max(vote_dict, key=vote_dict.get)
biggest_value = vote_dict[biggest_var_name]

if biggest_var_name == "var1":
    finalrounding.iloc[0, 1] += buffer_value

if biggest_var_name == "var2":
    finalrounding.iloc[0, 2] += buffer_value

if biggest_var_name == "var3":
    finalrounding.iloc[0, 3] += buffer_value

if biggest_var_name == "var4":
    finalrounding.iloc[0, 4] += buffer_value

if biggest_var_name == "var5":
    finalrounding.iloc[0, 5] += buffer_value

finalrounding.to_csv((f"{config.output_path}submission.csv"), index=False)
finalrounding.iloc[:, -6:].sum(axis=1)
display(finalrounding.head())

In [ ]:
# # SANITY CHECK TO CONFIRM PREDICTIONS SUM TO ONE
# finalrounding.iloc[:, -6:].sum(axis=1)